# Create Large Languge Model (LLM) object

You can use any LLM supported by LangChain or a third-party package. Or create your own class.

If you use OpenAI, then you need to set OPENAI_API_KEY as an environment variable.

In [1]:
from langchain_openai import ChatOpenAI
from rich.console import Console

console = Console()
llm = ChatOpenAI(model="gpt-4o-mini")

## Basic LLM prompts

The LLM object can respond to a prompt in plain text. 
While we are testing it out, let's test the knowledge cut off.

In [2]:
question = "Who will run in the 2024 US Presidential election?"
console.print(llm.invoke(question))

AIMessage(
    content='As of my last knowledge update in October 2023, several candidates had announced their intention to 
run in the 2024 U.S. Presidential election, including:\n\n### Democratic Party\n- **Joe Biden**: The incumbent 
President is running for re-election.\n\n### Republican Party\n- **Donald Trump**: The former President is running 
again after losing in 2020.\n- **Ron DeSantis**: Governor of Florida.\n- **Nikki Haley**: Former U.N. Ambassador 
and Governor of South Carolina.\n- **Vivek Ramaswamy**: Entrepreneur and author.\n- **Mike Pence**: Former Vice 
President.\n- **Tim Scott**: Senator from South Carolina.\n- **Chris Christie**: Former Governor of New 
Jersey.\n\n### Other Parties/Independents\nThere may also be candidates from third parties or independent 
candidates, but specific names and details about them would depend on the developments closer to the election 
date.\n\nFor the most current and accurate information, please check reliable news sources, as the political 
landscape can change rapidly leading up to the elections.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 218,
            'prompt_tokens': 19,
            'total_tokens': 237,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-4o-mini-2024-07-18',
        'system_fingerprint': 'fp_0705bf87c0',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-bce79ebc-cab0-4754-b9d0-cddb26f9dd69-0',
    usage_metadata={
        'input_tokens': 19,
        'output_tokens': 218,
        'total_tokens': 237,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

# Creating a chain

You might notice that the output of the LLM is a big complex `AIMessage` object. If we just want to extract the text as a string, we can use the `StrOutputParser` class.

You string a `chain` together with the pipe symbol `|`.

In [3]:
from langchain.schema.output_parser import StrOutputParser

parser = StrOutputParser()
chain = llm | parser
console.print(chain.invoke(question))

As of my last update in October 2023, several candidates had declared their intentions to run in the 2024 U.S. 
presidential election. On the Democratic side, President Joe Biden is expected to seek re-election. On the 
Republican side, notable candidates include former President Donald Trump, Florida Governor Ron DeSantis, and 
former United Nations Ambassador Nikki Haley, among others. There may also be candidates from third parties or 
independent runs, but specific details can change rapidly as the election approaches. For the most current 
information, I recommend checking reputable news sources.

## Prompt templates

To create a prompt template, you can use the `PromptTemplate` class. This class allows you to create a template with placeholders that you can fill in later.

It is useful when you want to create a prompt that you can reuse with different values.

In [4]:
from langchain_core.prompts.prompt import PromptTemplate

template = PromptTemplate.from_template("Write a couplet about {topic}.")
chain = template | llm | parser
console.print(chain.invoke("love"))

In whispered dreams where two hearts align,  
Love's gentle glow transforms the mundane to divine.

# Batch processing

You can also process multiple prompts in a batch. This is useful when you want to process multiple prompts at once.

Behind the scenes, the LangChain will run the prompts in parallel to speed up the processing.

In [5]:
topics = ["cats", "dogs", "birds"]
couplets = chain.batch(topics)
for topic, couplet in zip(topics, couplets):
    console.print(f"[bold]Couplet about {topic}:[/bold]\n{couplet}")

Couplet about cats:
In shadows sleek, with grace they roam,  
Whiskered whispers, they make our hearts their home.

Couplet about dogs:
With wagging tails and eyes so bright,  
Dogs bring pure joy, our hearts' delight.

Couplet about birds:
In skies of blue, the songbirds weave,  
A tapestry of melodies, hearts believe.

# Prompt Structure
The fine-tuning of Chat/Instruct LLMs often uses a `System` role, a `Human` role and an `AI` role. 
The `System` role is the one that provides the context. 
The `Human` role is the one that provides the input.
The `AI` role is the one that provides the output from the LLM.

In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a 5 year old kid."),
    ("human", "Write a couplet about {topic}."),
    ("ai", "Sure, here it is:"),
])

chain = template | llm | parser
console.print(chain.invoke("love"))

Love is like a warm, bright sun,  
It makes us smile and have lots of fun!

# Reading from an XML file
Let's build a chain that reads from a file, gets the abstract and summarises the content. 

First let's write a function to get an abstract from an XML file.


In [9]:
from pathlib import Path
from bs4 import BeautifulSoup

pubmed_dir = Path("pubmed")
def read_abstract(pmid:str) -> str:
    path = pubmed_dir / f"{pmid}.xml"
    xml_data = path.read_text()
    soup = BeautifulSoup(xml_data, 'xml')
    abstract_text = ' '.join(tag.get_text() for tag in soup.find_all('AbstractText'))
    return abstract_text

console.print(read_abstract("22883424"))

This meta-analysis compared the efficacy and safety of desvenlafaxine and venlafaxine at the Australian approved 
doses. A systematic literature search was conducted to identify all placebo-controlled studies of desvenlafaxine 
and venlafaxine in the treatment of major depression. The pivotal outcome measure used to assess comparative 
efficacy was the mean change in Hamilton Rating Scale for Depression-17 score from baseline. Tolerability and 
safety were compared by an evaluation of reported adverse events. Standard and Bayesian methods were used to 
conduct the indirect comparisons.                                       Findings               Using a mixed model 
repeated measures analysis, the pooled weighted mean difference for the mean change in Hamilton Rating Scale for 
Depression-17 score from baseline was -2.81 (-3.72, -1.91; p < 0.001) for desvenlafaxine and -2.61 (-3.17, -2.05; p
< 0.001) for venlafaxine. An indirect Bayesian analysis adjusted for baseline Hamilton Rating Scale for 
Depression-17 score showed no significant difference between the two treatments (weighted mean difference -0.27; 
-1.17, 0.65). A standard indirect comparison of any adverse events showed no significant difference between 
desvenlafaxine and venlafaxine (relative risk 1.01; 0.96, 1.06; p = 0.70 and risk difference -0.01; -0.05, 0.03; p 
= 0.59). Standard indirect comparisons of both nausea and drop-outs identified potential differences between 
treatments, with the risk difference analyses suggesting a trend in favor of desvenlafaxine (nausea: relative risk 
0.97; 0.77, 1.22; p = 0.80/RD -0.07; -0.12, -0.01; p = 0.02; and drop-outs due to adverse events: RR 0.86; 0.58, 
1.29; p = 0.48/RD -0.04; -0.08, 0.00; p = 0.06). Based on the results of this meta-analysis, desvenlafaxine was 
shown to be non-inferior to venlafaxine in terms of efficacy, and has an advantage in terms of less nausea.

# Use a function with a chain

You can often use use a function with a chain.

In [11]:
template = ChatPromptTemplate([
    ("system", "You are a bioinformatician with experience in statistics and data science."),
    ("human", "Read the abstract and summarize the methodology used in around 50 words:\n{abstract}"),
    ("ai", "Certainly, here is a summary of the methodology:"),
])
chain =  read_abstract | template | llm | parser
console.print(chain.invoke("22883424"))

A systematic literature search identified placebo-controlled studies comparing desvenlafaxine and venlafaxine for 
major depression. The primary efficacy measure was the mean change in Hamilton Rating Scale for Depression-17 
scores. Both standard and Bayesian methods were used for indirect comparisons of efficacy and safety, focusing on 
adverse events and tolerability.

In [17]:
pmids = [path.stem for path in pubmed_dir.glob("*.xml")]
summaries = chain.batch(pmids)
for pmid, summary in zip(pmids, summaries):
    console.print(f"[bold]Summary of {pmid}:[/bold]\n{summary}")

Summary of 22894826:
The study employed a search algorithm to identify 44 conserved genes across 22 cyanobacterial taxa. Phylogenetic 
analysis supported distinct positions for two basal species based on gene copy numbers. The correlation between 
ribosomal gene copies and terminal cell differentiation was examined, alongside comparisons of 16S rRNA variation 
with other eubacterial phyla using Bayesian phylogenetic inference.

Summary of 34351721:
In this open-label, adaptive, multiplatform trial, noncritically ill hospitalized Covid-19 patients were randomly 
assigned to receive either therapeutic-dose anticoagulation with heparin or usual-care thromboprophylaxis. The 
primary outcome, organ support-free days, was assessed using a Bayesian model, considering in-hospital death and 
days free of organ support. The trial concluded when superiority criteria for anticoagulation were met.

Summary of 22883424:
The meta-analysis involved a systematic literature search for placebo-controlled studies comparing desvenlafaxine 
and venlafaxine for major depression. Efficacy was assessed using the mean change in the Hamilton Rating Scale for 
Depression-17 score, while safety was evaluated through adverse events. Standard and Bayesian methods were employed
for indirect comparisons.

Summary of 34290493:
The proposed biclustering method employs a generalized Bayesian factor analysis framework, utilizing a sparse 
variational EM algorithm for parameter estimation. It integrates biological information represented as graphs and 
accommodates both continuous and discrete data types. Biclusters are identified through sparse estimates of factor 
loadings and conditional expectations of latent factors, demonstrating improved performance in multi-omics data 
analysis.

Summary of 28250509:
The study utilized the Complex Emergency Database to identify nutrition surveys in Ethiopia from 2000 to 2013, 
focusing on children aged 6-59 months with wasting defined by weight-for-height z-scores below -2. Data on conflict
and drought events were collected from public sources. A random-effects Bayesian meta-analysis synthesized findings
from 231 surveys, analyzing 175,607 children to estimate prevalence and impacts.

Summary of 31492041:
The study employed Bayesian genomic models to predict quantitative traits related to wood quality and tree growth 
in a 6-year-old Eucalyptus globulus breeding population. It assessed approximately 14,000 SNP markers and 3,000 
haplotypes, evaluating predictive ability across models. Bayesian ridge regression (BRR) and Bayes C showed the 
highest predictive ability, particularly for low-heritability traits when haplotype effects were included.

# Chain of Thought (CoT)

Let's create a CoT that reads from an XML file, gets the abstract and answers a question.

In [19]:
template = ChatPromptTemplate([
    ("system", "You are a knowledgeable bioinformatician specializing in statistics and data science."),
    ("human", 
        "Analyze the abstract and provide a step-by-step explanation of whether the article explicitly or implicitly uses Bayesian inference in its methodology. "
        "Focus on identifying key statistical methods and reasoning patterns:\n{abstract}"
    ),
    ("ai", "Certainly, here is my step-by-step analysis:"),
])
chain =  read_abstract | template | llm | parser
console.print(chain.invoke("22883424"))

To analyze whether the article explicitly or implicitly uses Bayesian inference in its methodology, we can break 
down the information provided in the abstract step-by-step. 

### Step 1: Identify the Purpose of the Study
The meta-analysis compares the efficacy and safety of two medications, desvenlafaxine and venlafaxine, for treating
major depression. 

### Step 2: Identify the Statistical Methods Used
The abstract mentions that both "standard and Bayesian methods were used to conduct the indirect comparisons." This
is a key indication that Bayesian inference is part of the methodology.

### Step 3: Examine the Findings
- **Mixed Model Repeated Measures Analysis**: The use of this method suggests a traditional frequentist approach to
analyzing the mean changes in Hamilton Rating Scale scores for both drugs.
- **Indirect Bayesian Analysis**: The mention of an "indirect Bayesian analysis" that adjusted for baseline 
Hamilton Rating Scale scores indicates an explicit use of Bayesian methodology. This contrasts with the standard 
methods where no Bayesian framework is used.

### Step 4: Analyze Specific Results
- The results of the Bayesian analysis indicated no significant difference between the two treatments (weighted 
mean difference of -0.27 with confidence intervals). This suggests that Bayesian methods were used to quantify 
uncertainty and infer probabilities regarding treatment efficacy.
- The standard indirect comparisons of adverse events were analyzed separately and did not show significant 
differences, indicating these comparisons were done using standard statistical methods without Bayesian inference.

### Step 5: Assess the Context of Bayesian Inference
Bayesian inference often involves updating the probability of a hypothesis as more evidence becomes available. In 
this context:
- The Bayesian approach in the meta-analysis likely involved using prior distributions and incorporating them with 
data from the studies included in the meta-analysis to derive posterior distributions for the treatment effects.
- The adjustment for baseline scores in the Bayesian analysis indicates a robust incorporation of prior knowledge 
into the model, which is a hallmark of Bayesian reasoning.

### Step 6: Conclusion
The article **explicitly** uses Bayesian inference in its methodology through the mention of "indirect Bayesian 
analysis." This is evidenced by the specific results reported from this analysis and the context in which the 
Bayesian methods were applied (adjusting for baseline scores). The findings from the Bayesian approach provided 
insights that were not obtained from the standard methods alone, thus highlighting the value of employing Bayesian 
statistics in this meta-analysis.

In summary:
- **Explicit Use of Bayesian Inference**: Yes, indicated by the phrase "indirect Bayesian analysis" in the 
abstract.
- **Key Statistical Methods**: Mixed model repeated measures for traditional analysis; Bayesian methods for 
indirect comparisons.
- **Reasoning Patterns**: Bayesian analysis provided a framework for incorporating prior information and assessing 
treatment effects, which is a significant aspect of the methodology.

# Multiple LLM calls
You can combine multiple LLM calls in a chain.

In [22]:
yes_no_template = ChatPromptTemplate([
    ("system", "You are a critical evaluator of text."),
    ("human", "Based on the following step-by-step analysis, did it affirmatively conclude that Bayesian inference was used? Answer with 'Y' or 'N' only:\n{analysis}"),
    ("ai", "Certainly, here is my response:"),
])

def yes_no_parser(text:str) -> bool:
    return text.strip().lower() == "y"

complex_chain = chain | yes_no_template | llm | StrOutputParser() | yes_no_parser
complex_chain.invoke("22883424")

True

# Forking chains
Now, let's combine that chain with another one to get a boolean value of whether or not the article used Bayesian inference.

In [25]:
from langchain_core.runnables import RunnablePassthrough

forked_chain = chain | dict(
    is_bayesian=RunnablePassthrough() | yes_no_template | llm | StrOutputParser() | yes_no_parser,
    justification=RunnablePassthrough(),
)

console.print(forked_chain.invoke("22883424"))

{
    'is_bayesian': True,
    'justification': 'To determine whether the article explicitly or implicitly uses Bayesian inference in its 
methodology, we will break down the abstract step-by-step and identify key statistical methods and reasoning 
patterns.\n\n### Step 1: Identify the Purpose of the Study\nThe study is a meta-analysis comparing the efficacy and
safety of two antidepressants: desvenlafaxine and venlafaxine. The primary outcome measure is the mean change in 
the Hamilton Rating Scale for Depression-17 score.\n\n### Step 2: Outline the Comparative Methods\nThe abstract 
mentions that both standard and Bayesian methods were used to conduct indirect comparisons. This is a critical 
point as it indicates that Bayesian methods were explicitly acknowledged in the study methodology.\n\n### Step 3: 
Analyze the Findings Related to Bayesian Analysis\n- **Indirect Bayesian Analysis**: The abstract states that an 
“indirect Bayesian analysis adjusted for baseline Hamilton Rating Scale for Depression-17 score” was performed. 
This suggests that Bayesian techniques were applied to adjust for baseline differences, which is typical in 
Bayesian analyses where prior information is incorporated.\n- **Results from Bayesian Analysis**: The result from 
this analysis shows no significant difference between the two treatments, with a weighted mean difference of -0.27 
and a confidence interval that includes 0 (-1.17, 0.65). This is indicative of the Bayesian approach, which often 
presents results in terms of posterior distributions and credible intervals.\n\n### Step 4: Compare to Standard 
Methods\n- The abstract also mentions that standard indirect comparisons were made for adverse events, which 
yielded non-significant results. This highlights a contrast between standard frequentist methods and Bayesian 
methods, providing a fuller picture of the analysis.\n  \n### Step 5: Examine the Statistical Terms Used\n- The use
of “relative risk” and “risk difference” in the context of standard comparisons indicates traditional statistical 
methods. However, the mention of the Bayesian analysis shows a deliberate application of alternative statistical 
reasoning that can incorporate prior knowledge and provide a different perspective on uncertainty.\n\n### Step 6: 
Assess Implicit Use of Bayesian Inference\nEven though the article explicitly states the use of Bayesian methods, 
it also implicitly suggests that Bayesian reasoning might influence the interpretation of results, especially given
the non-inferiority claim of desvenlafaxine to venlafaxine. The posterior probabilities and credible intervals 
derived from Bayesian analysis can often lead to different interpretations compared to p-values from frequentist 
methods.\n\n### Step 7: Conclusion\nIn summary, the article explicitly uses Bayesian inference in its methodology 
by stating that Bayesian methods were employed for indirect comparisons. The findings from the Bayesian analysis 
suggest no significant difference between the two treatments, which contrasts with the results from standard 
methods. Thus, the study employs both inference frameworks, but the explicit mention of Bayesian analysis confirms 
its role in the methodology.\n\n### Summary\n- **Explicit Use**: Bayesian methods mentioned for indirect 
comparisons.\n- **Key Findings**: Bayesian analysis showed no significant difference in efficacy.\n- **Contrasted 
with**: Standard statistical methods for adverse events.\n- **Implications**: Bayesian reasoning potentially 
affects interpretation of treatment efficacy and safety.\n\nThis stepwise breakdown confirms that the article 
clearly incorporates Bayesian inference as a fundamental part of its methodology while also highlighting the value 
of applying both Bayesian and standard statistical approaches.'
}

# Retrieval Augmented Generation (RAG)

Let's create a RAG that searches for a relevant articles and answers a question.

First we build the database.

In [28]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

embeddings_path = Path("embeddings.db")

def build_document(pmid:str) -> Document:
    return Document(
        page_content=read_abstract(pmid), 
        metadata=dict(pmid=pmid), # more metadata can be added from the XML file
    )

if Path(embeddings_path).exists():
    db = Chroma(persist_directory=str(embeddings_path), embedding_function=embeddings_model)
else:
    documents = [build_document(pmid) for pmid in pmids]
    db = Chroma.from_documents(
        documents=documents, 
        embedding=embeddings_model, 
        persist_directory=str(embeddings_path),
    )


# Using the database

You can use the database to search for relevant articles.

In [36]:
print(db.similarity_search("Australia", k=1))

[Document(metadata={'pmid': '22883424'}, page_content='This meta-analysis compared the efficacy and safety of desvenlafaxine and venlafaxine at the Australian approved doses. A systematic literature search was conducted to identify all placebo-controlled studies of desvenlafaxine and venlafaxine in the treatment of major depression. The pivotal outcome measure used to assess comparative efficacy was the mean change in Hamilton Rating Scale for Depression-17 score from baseline. Tolerability and safety were compared by an evaluation of reported adverse events. Standard and Bayesian methods were used to conduct the indirect comparisons.                                       Findings               Using a mixed model repeated measures analysis, the pooled weighted mean difference for the mean change in Hamilton Rating Scale for Depression-17 score from baseline was -2.81 (-3.72, -1.91; p < 0.001) for desvenlafaxine and -2.61 (-3.17, -2.05; p < 0.001) for venlafaxine. An indirect Bayesian 

# Building a simple RAG chain

In [35]:
context_count = 2
retriever = db.as_retriever(search_kwargs={"k": context_count})
rag_template = ChatPromptTemplate([
    ("system", "You are a bioinformatician with experience in statistics and data science."),
    ("human", 
        "Answer the question '{question}' based on the context which contains abstracts of journal articles.\n"
        "Context:\n{context}\nQuestion:\n{question}"
    ),
    ("ai", "Certainly, here is my answer:"),
])
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_template
    | llm
    | StrOutputParser()
)
console.print(rag_chain.invoke("What is the role of therapeutic anticoagulation for treating COVID-19?"))

The role of therapeutic anticoagulation in treating COVID-19, particularly in noncritically ill hospitalized 
patients, is to improve clinical outcomes by reducing the risk of complications associated with thrombosis and 
inflammation. A study indicated that administering therapeutic-dose anticoagulation with heparin, as opposed to 
usual-care pharmacologic thromboprophylaxis, significantly increased the probability of survival to hospital 
discharge without the need for cardiovascular or respiratory organ support. The trial demonstrated a 98.6% 
probability that therapeutic anticoagulation enhanced organ support-free days, with an adjusted odds ratio of 1.27.
Additionally, there was a 4.0 percentage point increase in the likelihood of survival without organ support in 
favor of therapeutic anticoagulation. While there is a small risk of major bleeding associated with this treatment 
(1.9% for therapeutic vs. 0.9% for thromboprophylaxis), the overall benefits in improving patient outcomes support 
the use of therapeutic anticoagulation in this patient population.

# Building a RAG chain which cites the source

In [38]:
def format_docs(docs):
    text = ""
    for doc in docs:
        text += f"\n\n[{doc.metadata['pmid']}] {doc.page_content}"
    
    return text

context_chain = retriever | format_docs

context_chain.invoke("What is the role of therapeutic anticoagulation for treating COVID-19?")

'\n\n[34351721] Thrombosis and inflammation may contribute to the risk of death and complications among patients with coronavirus disease 2019 (Covid-19). We hypothesized that therapeutic-dose anticoagulation may improve outcomes in noncritically ill patients who are hospitalized with Covid-19. In this open-label, adaptive, multiplatform, controlled trial, we randomly assigned patients who were hospitalized with Covid-19 and who were not critically ill (which was defined as an absence of critical care-level organ support at enrollment) to receive pragmatically defined regimens of either therapeutic-dose anticoagulation with heparin or usual-care pharmacologic thromboprophylaxis. The primary outcome was organ support-free days, evaluated on an ordinal scale that combined in-hospital death (assigned a value of -1) and the number of days free of cardiovascular or respiratory organ support up to day 21 among patients who survived to hospital discharge. This outcome was evaluated with the u

In [39]:
cite_rag_template = ChatPromptTemplate([
    ("system", "You are a bioinformatician with experience in statistics and data science."),
    ("human", 
        "Answer the question '{question}' based on the context which contains abstracts of journal articles.\n"
        "Cite the ID of the article in your response.\n"
        "Context:\n{context}\nQuestion:\n{question}"
    ),
    ("ai", "Certainly, here is my answer:"),
])
cite_rag_chain = (
    {"context": context_chain, "question": RunnablePassthrough()}
    | rag_template
    | llm
    | StrOutputParser()
)
console.print(cite_rag_chain.invoke("What is the role of therapeutic anticoagulation for treating COVID-19?"))

The role of therapeutic anticoagulation in treating COVID-19, particularly in noncritically ill hospitalized 
patients, is to improve outcomes by increasing the probability of survival to hospital discharge with reduced need 
for cardiovascular or respiratory organ support. A study indicated that therapeutic-dose anticoagulation with 
heparin significantly increased organ support-free days compared to usual-care pharmacologic thromboprophylaxis, 
with a probability of superiority reaching 98.6%. This approach may also favorably impact survival rates, 
particularly in patients with elevated d-dimer levels. However, it is important to note that there is a risk of 
major bleeding associated with therapeutic anticoagulation, occurring in 1.9% of patients compared to 0.9% in those
receiving thromboprophylaxis. 

This information is based on the findings from article ID [34351721].